# Pivoting

Il pivoting può essere eseguito con la funzione `pivot_table(values, index, columns, aggfunc="mean", fill_value=None, dropna=True, margins=False)`
*	values specifica l’attributo o gli attributi (se si passa una lista) da aggregare, tipicamente sono numerici;
*	index specifica l’attributo o gli attributi (se si passa una lista) da mantenere sulle righe, verranno definiti come nuovo indice;
*	columns attributo o attributi da cui prendere i valori per generare le nuove colonne;
*	aggfunc nel caso in cui ci siano più righe con lo stesso valore nell’attributo/i specificato in index usa questa funzione per aggregarli. Come per groupby si può passare un elenco di funzioni o un dictionary specificando per ogni valore (contenuto in values) che aggregazione eseguire;
*	dropna dice se considerare o meno i valori nulli nel processo di aggregazione, di default vengono ignorati;
*	fill_value se dopo l’aggregazione un valore risulta NaN, può essere impostato un valore in questo parametro da mettere come default.
*	margins: se impostato a True calcola il totale di riga e di colonna.


In [3]:
import pandas as pd

# Dati delle vendite
dati_vendite = [
    ("Modena", "Gennaio", 800),
    ("Bologna", "Gennaio", 1000),
    ("Modena", "Febbraio", 1500),
    ("Bologna", "Febbraio", 2000),
    ("Bologna", "Febbraio", 1000)
]

# Creazione del DataFrame
df = pd.DataFrame(dati_vendite, columns=["Città", "Mese", "Vendite"])


df

,Città,Mese,Vendite
0,Modena,Gennaio,800
1,Bologna,Gennaio,1000
2,Modena,Febbraio,1500
3,Bologna,Febbraio,2000
4,Bologna,Febbraio,1000


Le righe vengono raggruppate per città, nelle colonne vengono messi i valori del mese, il campo vendite sulla base del valore di riga/colonna (quindi dove c’è la stessa combinazione) viene aggregato facendo la somma e poi messo in quella cella.

In [4]:
df.pivot_table(values='Vendite', index='Città', columns='Mese', aggfunc='sum')

Mese,Febbraio,Gennaio
Città,,
Bologna,3000,1000
Modena,1500,800


Il pivoting può essere eseguito anche con il metodo `pd.crosstab`, a differenza di pivot_table consente di calcolare anche la percentuale sul totale.

La sintassi è `pd.crosstab(index, columns, values=None, aggfunc=None, margins=False, normalize=False)`
Dove:
*	Index: è una o più serie per cui raggruppare i dati per riga;
*	Columns: è una o più serie di valori per cui creare le colonne;
*	Values: è opzionale, sono i valori da inserire nelle celle, se omesso effettua un conteggio delle righe;
*	Aggfunc: è la funzione per cui aggregare i valori espressi in values. Può essere sum, average, count, min, max, etc.
*	Margins: se impostato a True calcola il totale di riga e di colonna;
*	Normalize: calcola la percentuale dei valori:
  *	In base al totale di riga se impostato a 'index'
  *	In base al totale di colonna se impostato a 'columns'
  *	In base al totale di righe e colonna (la somma di tutti i valori) se impostato a 'all' o True.


In [5]:
# Equivalente a prima
pd.crosstab(index=df['Città'], columns=df['Mese'], values=df['Vendite'],  aggfunc='sum')

Mese,Febbraio,Gennaio
Città,,
Bologna,3000,1000
Modena,1500,800


In [6]:
#Aggiungendo margins=True si ottengono i totali di riga e colonna
pd.crosstab(index=df['Città'], columns=df['Mese'], values=df['Vendite'],  aggfunc='sum', margins=True)

Mese,Febbraio,Gennaio,All
Città,,,
Bologna,3000,1000,4000
Modena,1500,800,2300
All,4500,1800,6300


In [14]:
#Aggiungendo normalize='columns' otteniamo la percentuale per colonna
pd.crosstab(index=df['Città'], columns=df['Mese'], values=df['Vendite'],  aggfunc='sum', margins=True, normalize='columns')

Mese,Febbraio,Gennaio,All
Città,,,
Bologna,0.666667,0.555556,0.634921
Modena,0.333333,0.444444,0.365079


In [16]:
#Aggiungendo normalize='index' otteniamo la percentuale per riga
pd.crosstab(index=df['Città'], columns=df['Mese'], values=df['Vendite'],  aggfunc='sum', margins=True, normalize='index')

Mese,Febbraio,Gennaio
Città,,
Bologna,0.750000,0.250000
Modena,0.652174,0.347826
All,0.714286,0.285714


In [13]:
#Aggiungendo normalize='all' otteniamo la percentuale in base al totale complessivo
pd.crosstab(index=df['Città'], columns=df['Mese'], values=df['Vendite'],  aggfunc='sum', margins=True, normalize='all')

Mese,Febbraio,Gennaio,All
Città,,,
Bologna,0.476190,0.158730,0.634921
Modena,0.238095,0.126984,0.365079
All,0.714286,0.285714,1.000000


# Unpivoting
L’unpivoting si esegue con il metodo `melt(id_vars, value_vars, var_name, value_name)` dove:
*	id_vars sono le colonne che non devono essere modificate;
*	value_vars sono le colonne che si vogliono trasporre in righe;
*	var_name è il nome della colonna che conterrà l’informazione su da quale colonna tra value_vars è stato estratto il valore.
*	value_name è il nome che verrà dato alla colonna dei valori che saranno estratti dalle colonne value_vars;


In [23]:
df1 = pd.crosstab(index=df['Città'], columns=df['Mese'], values=df['Vendite'],  aggfunc='sum')
df1.reset_index(inplace=True)
df1

Mese,Città,Febbraio,Gennaio
0,Bologna,3000,1000
1,Modena,1500,800


In [26]:
df1.melt(id_vars="Città", value_vars=["Gennaio", "Febbraio"], var_name="Mese", value_name='Vendite')

,Città,Mese,Vendite
0,Bologna,Gennaio,1000
1,Modena,Gennaio,800
2,Bologna,Febbraio,3000
3,Modena,Febbraio,1500
